In [8]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
import datetime as dt 

In [9]:
# 오늘 날짜 구하기
today = dt.date.today()

# 오늘 날짜 숫자 형식으로 출력
today_number = today.strftime("%Y%m%d")

days=int(today_number)
# print(today_number)

# # 이번 달 1일 날짜 구하기
# start_month = dt.date(today.year, today.month, 1)
# start_month_number = start_month.strftime("%Y%m%d")
# print(start_month_number)

# # 어제 날짜 구하기
# yesterday = today - dt.timedelta(days=1)
# yesterday_number = yesterday.strftime("%Y%m%d")
# print(yesterday_number)

In [10]:
#빈 리스트 생성
genie_chart=[]

#수집할 사이트 주소 설정
pages = ['1', '2'] 

start_date = dt.date(today.year, today.month, 1)
end_date = today - dt.timedelta(days=1)
delta = end_date - start_date

for i in range(delta.days + 1):
    date = (start_date + dt.timedelta(days=i)).strftime("%Y%m%d")
    
    for p in pages:
        
        #수집할 컨텐츠가 있는 웹 페이지의 주소
        url=f"https://www.genie.co.kr/chart/top200?ditc=D&ymd={date}&hh=09&rtm=N&pg={p}"
        # url

        #브라우저 버전 정보
        userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

        #접속 객체 생성
        session = requests.Session()

        #접속 객체에 부가정보 삽입하기
        session.headers.update({
            "Referer": "",
            "User-Agent": userAgent
        })

        #특정 웹 페이지 접속
        r=session.get(url)

        #접속에 실패한 경우
        if r.status_code != 200:
            #에러코드와 에러메시지 출력
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            #에러를 강제로 생성시킴
            raise Exception(msg)

        #인코딩 형식 지정하여 beautifulsoup 객체 생성
        r.encoding = "UTF-8"
        # print(r.text)
        soup=BeautifulSoup(r.text)
        # soup

        #필요한 내용 추출하기
        genie_list = soup.select('.title.ellipsis')
        # len(genie_list)

        for i in range(len(genie_list)):
            #곡명 추출
            songEL=soup.select('.title.ellipsis')
            # print(songEL)
            song=songEL[i].text.strip()
            # print(song)   
            
            #가수명 추출
            singerEL=soup.select('.artist.ellipsis')
            # print(singerEL)
            singer=singerEL[i].text.strip()
            # print(singer)

            #앨범명 추출
            albumEL=soup.select('.albumtitle.ellipsis')
            # print(albumEL)
            album=albumEL[i].text.strip()
            # print(album)

            #순위 추출
            rank= (int(p)-1)*50 + (1+i)
            # print(rank)

            if rank==1:
                days-=1

            #추출한 내용을 딕셔너리로 변환
            item={"순위":rank,"가수":singer,"곡명":song,"앨범명":album,"일수":str(days)}

            #딕셔너리를 미리 준비한 리스트에 원소로 추가
            genie_chart.append(item)
genie_chart

[{'순위': 1,
  '가수': '(여자)아이들',
  '곡명': '퀸카 (Queencard)',
  '앨범명': 'I feel',
  '일수': '20230627'},
 {'순위': 2,
  '가수': 'IVE (아이브)',
  '곡명': 'I AM',
  '앨범명': "I've IVE",
  '일수': '20230627'},
 {'순위': 3,
  '가수': 'aespa',
  '곡명': 'Spicy',
  '앨범명': 'MY WORLD - The 3rd Mini Album',
  '일수': '20230627'},
 {'순위': 4,
  '가수': 'IVE (아이브)',
  '곡명': 'Kitsch',
  '앨범명': "I've IVE",
  '일수': '20230627'},
 {'순위': 5,
  '가수': 'LE SSERAFIM (르세라핌)',
  '곡명': 'UNFORGIVEN (Feat. Nile Rodgers)',
  '앨범명': 'UNFORGIVEN',
  '일수': '20230627'},
 {'순위': 6, '가수': '지수 (JISOO)', '곡명': '꽃', '앨범명': 'ME', '일수': '20230627'},
 {'순위': 7,
  '가수': 'NewJeans',
  '곡명': 'Hype boy',
  '앨범명': "NewJeans 1st EP 'New Jeans'",
  '일수': '20230627'},
 {'순위': 8,
  '가수': 'NewJeans',
  '곡명': 'Ditto',
  '앨범명': "NewJeans 'OMG'",
  '일수': '20230627'},
 {'순위': 9,
  '가수': '윤하 (YOUNHA)',
  '곡명': '사건의 지평선',
  '앨범명': "YOUNHA 6th Album Repackage 'END THEORY : Final Edition'",
  '일수': '20230627'},
 {'순위': 10,
  '가수': '세븐틴 (SEVENTEEN)',
  '곡명': '손오공',
  '앨범명':

In [11]:
df=DataFrame(genie_chart)
df.set_index('순위',inplace=True) #순위를 우선으로 하기 위해 인덱스 값으로 적용
df.to_excel("genie_chart.xlsx")